# Autogen

A framework for building AI agents and applications

[microsoft.github.io/autogen/](microsoft.github.io/autogen/)

# Swarm

Swarm implements a team in which agents can hand off 
task to other agents based on their capabilities. 

It is a `multi-agent design pattern` first introduced by OpenAI in 
[Swarm](https://github.com/openai/swarm).


The key idea is to let agent delegate tasks to other agents using a special tool call, while
all agents share the same message context.

This enables agents to make local decisions about task planning, rather than
relying on a central orchestrator. 


## How Does It Work?

At its core, the `Swarm` team is a group chat where agents take turn to generate a response.

Participant agents broadcast their responses so all agents share the same message context.

At each turn, **the speaker agent is selected based on the most recent
`Hand off Message` message in the context.**

This naturally requires each agent in the team to be able to generate
a HandoffMessage to signal which other agents that it hands off to.

You can set the `handoffs` argument to specify which agents a specific agent can hand off to. 

The overall process can be summarized as follows:

1. Each agent has the ability to generate a Hand off Message
   to signal which other agents it can hand off to. 
2. When the team starts on a task, the first speaker agents operate on the task and make locallized decision about whether to hand off and to whom.
3. When an agent generates a Hand off Message, the receiving agent takes over the task with the same message context.
4. The process continues until a termination condition is met.


## Stock Research Example

![Stock Research](swarm_stock_research.svg)


This system is designed to perform stock research tasks by leveraging four agents:

- **Planner**: The central coordinator that delegates specific tasks to specialized agents based on their expertise. The planner ensures that each agent is utilized efficiently and oversees the overall workflow.
- **Financial Analyst**: A specialized agent responsible for analyzing financial metrics and stock data using tools such as `get_stock_data`.
- **News Analyst**: An agent focused on gathering and summarizing recent news articles relevant to the stock, using tools such as `get_news`.
- **Writer**: An agent tasked with compiling the findings from the stock and news analysis into a cohesive final report.

#### Workflow
1. The **Planner** initiates the research process by delegating tasks to the appropriate agents in a step-by-step manner.
2. Each agent performs its task independently and appends their work to the shared **message thread/history**. Rather than directly returning results to the planner, all agents contribute to and read from this shared message history. When agents generate their work using the LLM, they have access to this shared message history, which provides context and helps track the overall progress of the task.
3. Once an agent completes its task, it hands off control back to the planner.
4. The process continues until the planner determines that all necessary tasks have been completed and decides to terminate the workflow.

### Packages and Bing Helper Function 

In [8]:
!pip install azure-ai-inference[opentelemetry] azure-ai-projects==1.0.0b4 prompty[azure]==0.1.40, azure-identity==1.19.0
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import json
from dotenv import load_dotenv
import prompty
import prompty.azure
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool
from azure.ai.inference.prompts import PromptTemplate

load_dotenv()

def execute_research(instructions: str,prompty_name: str, feedback: str = "No feedback"):

    ai_project_conn_str = os.getenv("AZURE_LOCATION")+".api.azureml.ms;"+os.getenv("AZURE_SUBSCRIPTION_ID")+";"+os.getenv("AZURE_RESOURCE_GROUP")+";"+os.getenv("AZURE_AI_PROJECT_NAME")

    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=ai_project_conn_str,
    )

    # Load the prompt 
    prompt_template = PromptTemplate.from_prompty(file_path=os.getcwd() +f"/{prompty_name}.prompty")

    instructions = instructions
    feedback= feedback
    messages = prompt_template.create_messages(instructions=instructions, feedback=feedback)

    bing_connection = project_client.connections.get(
        connection_name='bing-connection'
    )
    conn_id = bing_connection.id

    # Initialize agent bing tool and add the connection id
    bing = BingGroundingTool(connection_id=conn_id)

    # Create agent with the bing tool and process assistant run
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="my-assistant",
            instructions=messages[0]['content'],
            tools=bing.definitions,
        )

        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=instructions,
        )
        print(f"Created message, ID: {message.id}")

        # # Create and process agent run in thread with tools
        # run = project_client.agents.create_stream(thread_id=thread.id, assistant_id=agent.id)

        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        
        print(f"Run finished with status: {run.status}")

        # Retrieve run step details to get Bing Search query link
        # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
        run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
        run_steps_data = run_steps['data']
        print(f"Agent created and now researching...")
        print('')

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # print(f"Messages: {messages}")
        research_response = messages.data[0]['content'][0]['text']['value']
        print(research_response)
        research = json.loads(research_response)
        print('research succesfully completed')
        return research




In [5]:
# Load the prompt 
prompt_template = PromptTemplate.from_prompty(file_path=os.getcwd() +f"/researcher.prompty")
messages = prompt_template.create_messages(instrcutions='find news about microsoft')
print(messages[0]['content'])

# Researcher Agent
You are a researcher tasked with obtaining stock market data for a given stock symbol using a web research tool. Return the results in the specified JSON format.

# Instructions

- For a given stock symbol (e.g., "AAPL") or stock name eg Tesla, use the web research tool to locate and extract relevant stock data.
- Collect the following key details:
  - **Price**: The most recent trading price of the stock.
  - **Volume**: The total number of shares traded during the current or most recent session.
  - **P/E Ratio**: The price-to-earnings ratio of the stock.
  - **Market Cap**: The company's market capitalization in abbreviated format (e.g., "700B" for 700 billion, "1.2T" for 1.2 trillion).
- Ensure all data is formatted accurately as numbers or strings, as required in the example.
- Make sure to ONLY return the final json object like in the example, nothing else.
Make sure to only return the json object and nothing else, just like in the examples below. Do not use th

### Tools

In [2]:
from typing import Any, Dict, List

async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    instructions = f"Get stock data for {symbol}"    
    research_response = execute_research(instructions=instructions, prompty_name="researcher")
    return research_response


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    instructions = f"{query}"    
    news = execute_research(instructions=instructions, prompty_name="news")
    return news

### Agents

In [3]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com"
    )

planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time. 
    Make sure to return the final report from the writer. 
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [4]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], 
    termination_condition=termination
)

task =  "Conduct market research for Nvidia stock and let me know if it's a good time to invest in it."
await Console(research_team.run_stream(task=task))

---------- user ----------
Conduct market research for Nvidia stock and let me know if it's a good time to invest in it.


/usr/local/python/3.12.1/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  model_result = await self._model_client.create(
/usr/local/python/3.12.1/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Both tool_calls and content are present in the message. This is unexpected. content will be ignored, tool_calls will be used.
  model_result = await self._model_client.create(


---------- planner ----------
[FunctionCall(id='call_Rt6DAuclorzHgMj2yAnyHZ0m', arguments='{}', name='transfer_to_financial_analyst')]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_Rt6DAuclorzHgMj2yAnyHZ0m')]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
Created agent, ID: asst_0YB8Rk7mnPK6VZnKFpOCViiz
Created thread, ID: thread_9MYqJQEbAp8suIpU2x6GtH0l
Created message, ID: msg_02soKHSylDzOM5v2ZTmyodZJ
Run finished with status: RunStatus.COMPLETED
Agent created and now researching...

{"price": 139.23, "volume": null, "pe_ratio": null, "market_cap": "700B"}
research succesfully completed
---------- financial_analyst ----------
[FunctionCall(id='call_1IMDSTZ4sGwYZIJ2cpuDn5p4', arguments='{"symbol":"NVDA"}', name='get_stock_data')]
---------- financial_analyst ----------
[FunctionExecutionResult(conte

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Conduct market research for Nvidia stock and let me know if it's a good time to invest in it.", type='TextMessage'), ToolCallRequestEvent(source='planner', models_usage=RequestUsage(prompt_tokens=196, completion_tokens=147), content=[FunctionCall(id='call_Rt6DAuclorzHgMj2yAnyHZ0m', arguments='{}', name='transfer_to_financial_analyst')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='planner', models_usage=None, content=[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', call_id='call_Rt6DAuclorzHgMj2yAnyHZ0m')], type='ToolCallExecutionEvent'), HandoffMessage(source='planner', models_usage=None, target='financial_analyst', content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', context=[], type='HandoffMessage'), ToolCallRequestEvent(source='financial_analyst', models_usage=RequestUsage(pr